# Introduction

TODO:
- Frame the business problem (why we care) and 3-5 potential use cases for this technology in the retail sector
- Define success in terms of 1-2 specific metrics (recommend using MAPE and RMSE)
- Provide a high-level overview of your approach
- Write a high-level overview of your data
    - Where it came from
    - How it was collected
    - Potential biases it may have
- When you're all finished, write a 1-paragraph executive summary for this engagement

# Import

TODO:
- Load pkl into a dask dataframe for parallelized processing
- Print .head() and .summary()

OPTIONAL:
- Distributively process your code using dask. See here: https://docs.dask.org/en/latest/setup/cloud.html

In [5]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from datetime import timedelta

import seaborn as sns

import xgboost as xgb
import lightgbm as lgb

import shap

# Import UDFs
import sys
sys.path.append('./src')
from utilities import *
from exploration import *
from feature_engineering import *
from modeling import *


In [2]:
pandas_df = pd.read_pickle("./data/raw_weekly_df.pkl")
dask_df = dd.from_pandas(pandas_df, npartitions=8)

In [3]:
(pandas_df.info(memory_usage="Deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 7 columns):
 #   Column    Dtype         
---  ------    -----         
 0   dept_id   category      
 1   cat_id    category      
 2   item_id   category      
 3   state_id  category      
 4   store_id  category      
 5   datetime  datetime64[ns]
 6   sales     float32       
dtypes: category(5), datetime64[ns](1), float32(1)
memory usage: 117.5 MB


In [9]:
#Unable to allocate 365. MiB for an array with shape (7, 6841121) and data type float64 error?
compress_dataframe(pandas_df)

TypeError: arg must be a list, tuple, 1-d array, or Series

# Data Exploration & Transformation

TODO:
- Check for outliers and missing data
- Check for duplicates
- Check datatypes and comment on whether they make sense
- Plot univariate / bivariate graphs to better understand your data, especially for your target variable, in seaborn
- Standardize, normalize, or log your target variable
- Split your data into training, testing, and holdout sets and explain why this step is important

In [ ]:
pandas_df.duplicated()

- shows that none of the rows are duplicated

In [ ]:
pandas_df.loc[pandas_df.duplicated() != False]

 - 6841121 cells of data are in the data set
 - the mean of the sales from all pieces of data is 9.60301
 - the range of the sales from the data is 0 to 4220
 - the distribution of the data shows 25% of the sales data having a value below 0, 50% of the sales data having a value below 3 (median), and 75% of the sales data having a value below 9
 - the sales data is skewed right since the mean is greater than the median

In [3]:
pandas_df.describe()

,sales
count,6.841121e+06
mean,9.526814e+00
std,2.519556e+01
min,0.000000e+00
25%,0.000000e+00
50%,3.000000e+00
75%,9.000000e+00
max,4.220000e+03


The majority of sales are between 0 and 10, with most sales falling at 0.

In [ ]:
#TODO source files shouldn't call functions; move to .main
univariate_sales_histogram(pandas_df, "sales")

Out of all three states, CA has the most number of sales (around 700,000 sales at 0.0) as compared to Wisconsin and Texas.
This makes sense since California has the largest population compared to the other two.

In [ ]:
bivariate_sales_by_state(pandas_df, "sales")


In the previous observation, CA was observed to have the most number of 0.0 sales compared to the other two states. However, through this histogram, we see that CA actually has more stores than Texas and Wisconsin. All the graphs of the stores seem pretty similar, but if you observe closely, CA_1, CA_2, CA_3 (especially CA_3), CA_4, TX_1, TX_2 all have sales between 30 and 50 as well, unlike the other stores. This makes sense as well since CA has the largest population compared to the other two states, and compared to Wisconsin, Texas has a larger population."""

In [ ]:
bivariate_sales_by_store(pandas_df, "sales")

In [ ]:
#TODO split these up into markdown cells (see examples above)
sales_boxplot(pandas_df, "sales")
"""The max sales value, 4220 is clearly an outlier. There also seem to be 6 other outliers since they are separate. All the outliers are larger not smaller."""
"""California has this outlier which is the largest value; this makes sense since California has a larger amount of people who contribute to these larger outliers occassionally."""
sales_box_plot_by_state_id(pandas_df, "sales")
"""California has the largest value, 4220, which is also its largest outlier. California has 2 outliers that are 
at least 1000 more than the maximum in its data. Texas and Wisconsin have outliers that are smaller than 2000. Wisconsin has the least number of outliers."""
sales_box_plot_by_store_id(pandas_df, "sales")
"""Every store's outliers are beneath 2000 except for CA_3, all of whose outliers crossed 2000.
Since the range for CA_3 is higher than that of every other store, it indicates that CA_3 had the most sales."""
"""This makes sense since California has the largest population, so there will be more sales and larger outliers."""



In [ ]:
heatmap_to_see_missing_data(pandas_df)
"""The heatmap has a display column for each series in a pandadataframe, and will have horizontal rows within each column wherever there are missing values/data (NaN). In this case, there aren't any missing data so the heatmap is empty."""

# Feature Engineering

TODO:
- Discretize any categorical features
- Develop 3+ new time-related features and 3+ new product-related features as functions in a script called 'feature_engineering.py' in ./src
    - Time-related features: MonthYear, holiday flags, etc.
    - Product-related features: average number of items sold per month in each store (frequency), last time product sold in the store (recency), etc. 

In [3]:
standardize(pandas_df,'sales')


0         -0.259046
1         -0.020909
2         -0.020909
3         -0.060599
4          0.177539
             ...   
6841116   -0.378115
6841117   -0.378115
6841118   -0.378115
6841119   -0.378115
6841120   -0.378115
Name: sales, Length: 6841121, dtype: float32

In [4]:
unstandardize(standardize(pandas_df,'sales'),'sales')

0           3.0
1           9.0
2           9.0
3           8.0
4          14.0
           ... 
6841116     0.0
6841117     0.0
6841118     0.0
6841119     0.0
6841120     0.0
Name: sales, Length: 6841121, dtype: float32

In [6]:
stat_feature(pandas_df,'state_id','sales',['mean','min','max'])

,mean,min,max
state_id,,,
CA,10.585984,0.0,4220.000488
TX,9.033415,0.0,1871.000122
WI,8.882002,0.0,1755.000122


In [7]:
cum_stat(pandas_df,'state_id','sales')

,cumsum
0,NaN
1,3.0
2,12.0
3,21.0
4,29.0
...,...
6841116,65174088.0
6841117,65174088.0
6841118,65174088.0
6841119,65174088.0


In [8]:
detrend_feature(pandas_df,'state_id','sales')

0          NaN
1          6.0
2          0.0
3         -1.0
4          6.0
          ... 
6841116    0.0
6841117    0.0
6841118    0.0
6841119    0.0
6841120    0.0
Name: sales, Length: 6841121, dtype: float32

# Modeling

TODO: 
- Write a function to run your lightgbm and xgboost models on your training data and output your in-sample and out-of-sample error RMSE and MAPE
- Use your function to 'tune' both algorithms by passing in different combinations of hyperparameters (max depth, subsampling, etc.) to optimize for OOS RMSE
- Plot your OOS RMSE on the y-axis and your various hyperparameter combinations on the x-axis to select the best combination
- Retrain your models on your train + test data and measure your IS and OOS error metrics on your holdout data 

OPTIONAL:
- Validate that your results are accurate using k-fold cross-validation. Note that this step is extremely important, but can be time-intensive; including it here as optional to be conscious of our time constraints.

# Interpretation

TODO:
- Create a SHAP plot for our best model and interpret its results
